In [ ]:
#pip install -U kaleido

In [1]:
import pandas as pd
import pickle
import plotly #pip install -U kaleido required
import plotly.graph_objects as go
import plotly.express as px

In [2]:
# read data_dict
infile = open('../ProtacKB/data/evolvus/Aug23/evo_dict_run_1.pkl', 'rb')
data_dict = pickle.load(infile)
infile.close()

# read node_dict
infile = open('../ProtacKB/data/evolvus/Aug23/evo_dict_run_9.pkl', 'rb')
node_dict = pickle.load(infile)
infile.close()

del node_dict['iGLUE']

In [3]:
entity = {}
for item in node_dict:
    print(len(node_dict[item]))
    entity.update({item:len(node_dict[item])})
    

130
14
11906
4464
2477
1221
82
3525
68
2077
673
240
364
517
53


In [4]:
entity

{'Protein': 130,
 'E3 Ligase': 14,
 'PROTAC': 11906,
 'Linker': 4464,
 'Warhead': 2477,
 'E3 Binder': 1221,
 'Disease': 82,
 'Degrader': 3525,
 'Shuttle Protein': 68,
 'Molecular Glue': 2077,
 'Reactome': 673,
 'Function': 240,
 'Biological Process': 364,
 'AdverseEffect': 517,
 'MOA': 53}

In [8]:
def KG_namespace_plot(final_kg,filename):
    
    #df = dict(final_kg.count.namespaces())
    df = pd.DataFrame([final_kg]).T
    df = df.reset_index()
    df.columns = ['Entity','Numbers']
    
    fig = px.bar(df,x='Entity',y='Numbers',color='Entity')
    fig.update_layout(title_text='PROTACKB entity in numbers', title_x=0.5)
    
    path_img = 'images/' + filename + '_namespace.png'
    fig.write_image(path_img,scale=5)
    
    path_html = 'images/' + filename + '_namespace.html'
    plotly.offline.plot(fig, filename= path_html)


In [9]:
KG_namespace_plot(entity,'KG')

In [19]:
xx = []
for item in data_dict:
    #print(item['PD_Subtype'])
    if item['PD_Subtype'] != None:
        #if item['PD_Subtype']['PD_Subtype'] not in xx:
        #print(item['PD_Subtype']['PD_Subtype'])
        #print(item['PD']['SMILES'])
        xx.append(item['PD_Subtype']['PD_Subtype'])

#print(xx)

from collections import Counter

def count_frequency(lst):
    return Counter(lst)

frequency = count_frequency(xx)

for item, count in frequency.items():
    print(f"{item}: {count}")

PROTAC: 11770
Molecular glue: 2148
AUTAC: 2
PROTAC - IMiD: 2
homo-PROTAC: 4
Shuttle proteins: 73
Dual PROTAC: 8
Heterobifunctional Degraders: 3596


In [20]:
frequency

Counter({'PROTAC': 11770,
         'Heterobifunctional Degraders': 3596,
         'Molecular glue': 2148,
         'Shuttle proteins': 73,
         'Dual PROTAC': 8,
         'homo-PROTAC': 4,
         'AUTAC': 2,
         'PROTAC - IMiD': 2})

In [21]:
edge_df = pd.read_csv('../ProtacKB/data/evolvus/Aug23/edge_df_6thNov.csv')

In [22]:
edge_df.head()

,Subject,Predicate,Object,EdgeCol
0,AR,isInvolvedin,HSP90 chaperone cycle for steroid hormone rece...,(_18196)-[:isInvolvedin {}]->(_18197)
1,AR,isInvolvedin,Nuclear Receptor transcription pathwa,(_18196)-[:isInvolvedin {}]->(_18198)
2,AR,isInvolvedin,SUMOylation of intracellular receptor,(_18196)-[:isInvolvedin {}]->(_18199)
3,AR,isInvolvedin,Activated PKN1 stimulates transcription of AR ...,(_18196)-[:isInvolvedin {}]->(_18200)
4,AR,isInvolvedin,Ub-specific processing protease,(_18196)-[:isInvolvedin {}]->(_18201)


In [74]:
edge_df = edge_df.loc[edge_df['Predicate'] == 'hasE3Ligase']

In [75]:
e3lig = list(edge_df['Object'])

In [76]:
len(e3lig)

1224

In [83]:
frequency = count_frequency(e3lig)

for item, count in frequency.items():
    print(f"{item}: {count}")

VHL: 333
CRBN: 673
cIAP1: 159
DCAF15: 46
MDM2: 9
UBR1: 2
RNF4: 1
RNF114: 1


In [84]:
for i, n in enumerate(e3lig):
    if n == 'Cereblon':
        e3lig[i] = 'CRBN'
        
    if n == 'Mdm2':
        e3lig[i] = 'MDM2'
        
    if n == 'cIAP' or n == 'Iap':
        e3lig[i] = 'cIAP1'
        
    if n == 'Ubr1':
        e3lig[i] = 'UBR1'

In [90]:
df = dict(frequency)
df

{'VHL': 333,
 'CRBN': 673,
 'cIAP1': 159,
 'DCAF15': 46,
 'MDM2': 9,
 'UBR1': 2,
 'RNF4': 1,
 'RNF114': 1}

In [89]:
pd.DataFrame([frequency],columns=['a','b'])

,a,b
0,NaN,NaN


In [ ]:
#drugType figure
df_drugType = df.groupby(['drugType'])['drugType'].count().reset_index(name='count')

fig = px.pie(df_drugType, values='count', names='drugType')
fig.update_layout(title_text='Types of drugs', title_x=0.45)

path_img = '../streamlit_ITMP/images/' + filename + '_drugType.png'
fig.write_image(path_img,scale=5)

path_html = '../streamlit_ITMP/images/' + filename + '_drugType.html'
plotly.offline.plot(fig, filename= path_html)

In [98]:
import plotly.graph_objects as go

# Your data
data = df

# Create lists for labels and values
labels = list(data.keys())
values = list(data.values())

# Create the pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])

# Update layout
fig.update_layout(
    title_text="Proportion of E3 Ligase specific E3 Binders", title_x=0.5,
    # Add any other layout customizations here
)

path_img = 'images/' + 'E3Ligase.png'
fig.write_image(path_img,scale=5)

path_html = 'images/' + 'E3Ligase.html'
plotly.offline.plot(fig, filename= path_html)


# Show the chart
fig.show()
